# import library

In [1]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


## Data Loading

In [2]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Split dataset menjadi training, test, dan validation. Selain itu, kita juga perlu mendefinisikan kolom target.

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [4]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])

Kemudian bungkus dataframe dengan tf.data agar memungkinkan untuk menggunakan feature column sebagai jembatan dari dataframe menjadi fitur yang digunakan untuk train model. Jika file CSV nya sangat besar, kita harus menggunakan tf.data untuk membaca file tersebut dari harddisk.

In [5]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

Numeric column: Numeric column merupakan jenis feature column yang paling sederhana dan digunakan untuk merepresentasikan sebuah fitur dengan value yang apa adanya. Masukkan ke TensorFlow harus berupa angka dan numeric column sudah merupakan angka sehingga tidak perlu diubah lagi.

In [6]:
import tf_keras
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = tf_keras.layers.DenseFeatures([feature_column])
  print(feature_layer(example_batch).numpy())

Bucketized column: Terkadang kita memiliki dataset numerik yang memiliki value beragam dengan range cukup jauh. Daripada memasukkan setiap value ke dalam numeric column, kita dapat menggunakan bucketized column untuk membagi value-value tersebut.



In [7]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


Categorical column: Kita tidak dapat memasukkan value String ke dalam TensorFlow. Oleh karena itu, categorical column digunakan untuk mengubah value String pada dataset (pada dataset ini contohnya cut dan clarity) menjadi angka.

In [8]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])
 
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]


In [9]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[-0.200885   -0.23245567  0.01604145 -0.32985896  0.4289783   0.05875478]
 [ 0.5022464  -0.04362078 -0.22022763 -0.5821308   0.38828218 -0.09555344]
 [-0.14468989 -0.15773247  0.6144009  -0.32348272 -0.26165166 -0.48791212]
 [-0.27176982  0.2439798  -0.3369739  -0.18909211 -0.1569601   0.19296476]
 [-0.17828286  0.01535319 -0.62251925  0.10219855  0.13954568  0.20376205]
 [-0.27176982  0.2439798  -0.3369739  -0.18909211 -0.1569601   0.19296476]
 [-0.27176982  0.2439798  -0.3369739  -0.18909211 -0.1569601   0.19296476]
 [-0.27176982  0.2439798  -0.3369739  -0.18909211 -0.1569601   0.19296476]
 [-0.200885   -0.23245567  0.01604145 -0.32985896  0.4289783   0.05875478]
 [-0.17828286  0.01535319 -0.62251925  0.10219855  0.13954568  0.20376205]]


In [10]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]


In [11]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [12]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))
 
#membuat feature layer
feature_layer = tf_keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [13]:
#create, compile, and train the model
model = tf_keras.Sequential([
  feature_layer,
  tf_keras.layers.Dense(128, activation='relu'),
  tf_keras.layers.Dense(128, activation='relu'),
  tf_keras.layers.Dropout(.1),
  tf_keras.layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf_keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.2111 - accuracy: 0.9079 - val_loss: 0.1486 - val_accuracy: 0.9362
Epoch 2/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.2111 - accuracy: 0.9079 - val_loss: 0.1486 - val_accuracy: 0.9362
Epoch 2/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1580 - accuracy: 0.9334 - val_loss: 0.1820 - val_accuracy: 0.9232
Epoch 3/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1580 - accuracy: 0.9334 - val_loss: 0.1820 - val_accuracy: 0.9232
Epoch 3/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1523 - accuracy: 0.9356 - val_loss: 0.1549 - val_accuracy: 0.9326
Epoch 4/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1523 - accuracy: 0.9356 - val_loss: 0.1549 - val_accuracy: 0.9326
Epoch 4/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1506 - accuracy: 0.9368 - val_loss: 0.1513 - val_accuracy: